Here we are preprocessing images from the raw/input, turning them grayscale, resizing(1024 -> 128),

In [ ]:
import os
import cv2

# here are the paths to input/output folders, you can update, but actually don't have to ;>
folders = [
    (
        "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/raw/input",
        "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
    ),
    (
        "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/raw/style/blur",
        "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/blur"
    ),
    (
        "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/raw/style/outline",
        "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/outline"
    ),
    (
        "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/raw/style/poster",
        "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/poster"
    ),
    (
        "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/raw/style/night_vis",
        "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/night_vis"
    ),
]

valid_ext = {".jpg", ".jpeg", ".png", ".bmp", ".tiff"}

RESIZE_TO = (256, 256)   # RGB models need consistent size


def preprocess_image(path, resize_shape=None):
    img = cv2.imread(path)

    if img is None:
        return None
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Resize if desired
    if resize_shape is not None:
        img = cv2.resize(img, resize_shape)

    return img


for input_dir, output_dir in folders:

    if not os.path.exists(input_dir):
        print(f"Skipping missing folder: {input_dir}")
        continue

    os.makedirs(output_dir, exist_ok=True)

    print(f"\nProcessing folder:\n  INPUT:  {input_dir}\n  OUTPUT: {output_dir}")

    for filename in os.listdir(input_dir):
        ext = os.path.splitext(filename)[1].lower()
        if ext not in valid_ext:
            continue

        src_path = os.path.join(input_dir, filename)
        out_path = os.path.join(output_dir, filename)

        processed_img = preprocess_image(src_path, resize_shape=RESIZE_TO)

        if processed_img is None:
            print(f"Warning: Could not read {src_path}")
            continue

        # Save as PNG or JPEG (OpenCV saves RGB fine)
        cv2.imwrite(out_path, cv2.cvtColor(processed_img, cv2.COLOR_RGB2BGR))
        print(f"Saved: {filename}")

print("\nPreprocessing complete.")



Processing folder:
  INPUT:  /Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/raw/input
  OUTPUT: /Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input
Saved: grape.png
Saved: orange.png
Saved: apple.png
Saved: banana.png
Saved: kiwi.png

Processing folder:
  INPUT:  /Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/raw/style/blur
  OUTPUT: /Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/blur
Saved: banana_blur.png
Saved: kiwi_blur.png
Saved: orange_blur.png
Saved: grape_blur.png
Saved: apple_blur.png

Processing folder:
  INPUT:  /Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/raw/style/comix
  OUTPUT: /Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/comix
Saved: banana_comix.png
Saved: apple_comix.png
Saved: orange_comix.png
Saved: grape_comix.png
Saved: kiwi_comix.png

Processing folder:
  INPUT:  /Users/amayako